In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install scikit-plot

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn import preprocessing
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation, Embedding
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn.metrics import plot_confusion_matrix, classification_report
import matplotlib.pyplot as plt
import scikitplot as skplt
import h5py
from sklearn.model_selection import KFold,cross_val_score
kfold=KFold(5,True,3)
cvscores = []

In [ ]:
ds= pd.read_csv('/content/drive/MyDrive/code/dataset/df_spark.csv')
ds.shape
ds = ds.drop(columns="Unnamed: 0")
ds.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,7.0,21.0,30.0,2.0,3.0,30.0,2.0,3.0,59.0,7.0,2.0,0.0
1,7.0,32.0,70.0,2.0,4.0,68.0,2.0,4.0,134.0,7.0,2.0,0.0
2,7.0,15.0,1.0,2.0,2.0,0.0,2.0,2.0,0.0,7.0,2.0,0.0
3,7.0,33.0,77.0,2.0,7.0,73.0,2.0,7.0,144.0,7.0,2.0,0.0
4,7.0,55.0,78.0,3.0,7.0,74.0,3.0,7.0,146.0,8.0,2.0,0.0


In [ ]:
y = ds.iloc[:,0].values
X = ds.iloc[:,1:].values

from sklearn.preprocessing import label_binarize

y = label_binarize(y, classes=[0, 1, 2, 3, 4, 5, 6, 7])
n_classes = y.shape[1]
n_classes

8

In [ ]:
sc = StandardScaler()
for train, test in kfold.split(X,y):

    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
    X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

    model = Sequential()
    model.add(LSTM(16,input_dim=11, return_sequences=True))  
    model.add(Dropout(0.4))
    model.add(LSTM(16, return_sequences=True))  
    model.add(Dropout(0.4))
    model.add(LSTM(16, return_sequences=False))  
    model.add(Dropout(0.4))
    model.add(Dense(8))
    model.add(Activation('softmax'))
    #model.summary()

    model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy','mae'])
    history = model.fit(X_train, y_train, epochs = 10, batch_size=64, verbose=1,shuffle=False)
    scores = model.evaluate(X_test, y_test, verbose=1)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100)) 
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Epoch 1/10
4475/4475 [==============================] - 21s 5ms/step - loss: 0.1553 - accuracy: 0.9770 - mae: 0.0163
Epoch 2/10
4475/4475 [==============================] - 21s 5ms/step - loss: 0.0745 - accuracy: 0.9809 - mae: 0.0075
Epoch 3/10
4475/4475 [==============================] - 21s 5ms/step - loss: 0.0601 - accuracy: 0.9842 - mae: 0.0063
Epoch 4/10
4475/4475 [==============================] - 25s 6ms/step - loss: 0.0542 - accuracy: 0.9853 - mae: 0.0057
Epoch 5/10
4475/4475 [==============================] - 23s 5ms/step - loss: 0.0507 - accuracy: 0.9858 - mae: 0.0055
Epoch 6/10
4475/4475 [==============================] - 21s 5ms/step - loss: 0.0487 - accuracy: 0.9863 - mae: 0.0053
Epoch 7/10
4475/4475 [==============================] - 21s 5ms/step - loss: 0.0471 - accuracy: 0.9867 - mae: 0.0052
Epoch 8/10
4475/4475 [==============================] - 21s 5ms/step - loss: 0.0456 - accuracy: 0.9871 - mae: 0.0050
Epoch 9/10
4475/4475 [==============================] - 21s 5ms/

In [ ]:
target_names = ['DoS', 'Probing',	'MaliciousControl', 'MaliciousOperation', 'Scan', 'Spying', 'WrongSetup', 'Normal']

In [ ]:
### train_accuracy
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error,roc_auc_score,cohen_kappa_score)
import sklearn.metrics as metrics

expected = y_train
predicted = model.predict(X_train)
data = classification_report(expected,np.round(predicted), target_names=target_names)
print(data)

print("Accuracy_Train:",metrics.accuracy_score(expected, np.round(predicted)))
print("Precision_Train:",metrics.precision_score(expected, np.round(predicted), average='weighted'))
print("Recall_Train:",metrics.recall_score(expected, np.round(predicted), average='weighted'))
print("f1-score_Train:",metrics.f1_score(expected, np.round(predicted), average='weighted'))
print("ROC_AUC_Train:",metrics.roc_auc_score(expected, predicted,multi_class="ovr",average='weighted'))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                    precision    recall  f1-score   support

               DoS       0.98      0.66      0.79      4622
           Probing       0.00      0.00      0.00       257
  MaliciousControl       0.81      0.97      0.88       706
MaliciousOperation       1.00      1.00      1.00       645
              Scan       0.80      0.66      0.72      1249
            Spying       0.00      0.00      0.00       409
        WrongSetup       0.00      0.00      0.00       100
            Normal       0.99      1.00      1.00    278364

         micro avg       0.99      0.99      0.99    286352
         macro avg       0.57      0.54      0.55    286352
      weighted avg       0.99      0.99      0.99    286352
       samples avg       0.99      0.99      0.99    286352

Accuracy_Train: 0.9897049784880148


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision_Train: 0.988188655456589
Recall_Train: 0.9897049784880148
f1-score_Train: 0.9883823836771932
ROC_AUC_Train: 0.9848647927120724


In [ ]:
### test accuracy
expected = y_test
predicted = model.predict(X_test)
data = classification_report(expected,np.round(predicted), target_names=target_names)
print(data)
print("Accuracy:",metrics.accuracy_score(expected, np.round(predicted)))
print("Precision:",metrics.precision_score(expected, np.round(predicted), average='weighted'))
print("Recall:",metrics.recall_score(expected, np.round(predicted), average='weighted'))
print("f1-score:",metrics.f1_score(expected, np.round(predicted), average='weighted'))
print("ROC_AUC:",metrics.roc_auc_score(expected, predicted,multi_class="ovr",average='weighted'))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                    precision    recall  f1-score   support

               DoS       0.99      0.63      0.77      1158
           Probing       0.00      0.00      0.00        85
  MaliciousControl       0.79      0.95      0.86       183
MaliciousOperation       1.00      1.00      1.00       160
              Scan       0.76      0.64      0.69       298
            Spying       0.00      0.00      0.00       123
        WrongSetup       0.00      0.00      0.00        22
            Normal       0.99      1.00      1.00     69560

         micro avg       0.99      0.99      0.99     71589
         macro avg       0.57      0.53      0.54     71589
      weighted avg       0.99      0.99      0.99     71589
       samples avg       0.99      0.99      0.99     71589

Accuracy: 0.9886435066839878
Precision: 0.9868842571954886


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Recall: 0.9886435066839878
f1-score: 0.9870967038000839
ROC_AUC: 0.982098719591154
